In [1]:
import thinkbayes
import seaborn as sns
import pandas as pd

In [2]:
%pylab inline
pylab.rcParams['figure.figsize'] = (16, 8)

Populating the interactive namespace from numpy and matplotlib


# 근사 베이지안 계산
___
## 1. Variability Hypothesis (변이가설?)

- 다음은 변이가설의 예이다
>"males have a greater range of ability than females,
especially in intelligence. In other words, he believed that
most geniuses and most mentally retarded people are men. Because
he considered males to be the ’superior animal,’ Meckel
concluded that females’ lack of variation was a sign of inferiority."

- 이는 남성이 여성보다 성질의 편차가 크고(키의 크고 작음, 아이큐의 높고 낮음), 이는 남성의 우월함의 표시라는 조안 미켈의 가설이다
- 이와 관련해서 최근 미국의 남녀 성인이 키를 직접 입력한 데이터를 살펴 보니 다음과 같음을 볼 수 있었음
> - 남성의 평균 키는 178cm고, 여성의 평균 키는 163cm로 평균적으로 남성이 더 큼
> - 남성의 표준편차는 7.7cm고, 여성의 표준편차는 7.3cm로 절대적인 가변성은 남성이 더 높음
> - 하지만 이종 그룹간의 비교를 위한 변동계수(편차/평균)<sup>coefficient of variation, CV</sup>를 살펴보면 남성은 0.0433이고, 여성은 0.0444로 확인

- 물론 숫자들이 매우 비슷하여 데이터셋이 Variability Hypothesis를 증명하기에는 빈약하다고 볼 수 있음
- 하지만 베이지안 메서드를 사용하여 결론을 좀 더 명확하게 할 수는 있음
- 이는 다음 과정을 따름
> 1. 우선 간단한 구현으로 시작 (1,000개 이하의 데이터셋에서 작동)
> 2. 로그변환하여 확률을 계산하려면 전체 데이터를 표준화해야 하지만, 그럼 계산 속도가 느려짐
> 3. 보통 ABC로 알려져 있는 근사 베이지안을 통해 속도를 높일 예정

___
## 2. 평균과 표준편차

- 이번에도 이전 장과 마찬가지로, 결합분포를 통해 가우시안 분포의 변수인 mu와 sigma를 동시에 추정
- `Height` suite을 생성하고, suite은 각 (mu, sigma)쌍으로 구성:

```python
class Height(thinkbayes.Suite, thinkbayes.Joint):
    
    def __init__(self, mus, sigmas):
        thinkbayes.Suite.__init__(self)
        
        pairs = [(mu, sigma)
                for mu in mus
                for sigma in sigmas]
        
        thinkbayes.Suite.__init__(self, pairs)
        ```

- `mus`는 mu의 값들이고, `sigmas`는 simga 값들로 이루어진 객체로 사전분포는 균등분포를 가정
- 우도함수는 mu와 sigma가 주어졌을 때 특정 값 x에 대한 우도를 계산 (`EvalGaussianPdf`)
```python
#class Height
    
    def Likelihood(self, data, hypo):
        x = data
        mu, sigma = hypo
        like = thinkbayes.EvalGaussianPdf(x, mu, sigma)
        return like
```

- PDF를 계산하면 확률 밀도가 나오고, 이를 특정 범위에 대해 적분해야 함
- 하지만 본 케이스에서 필요한 것은 확률의 비율이므로, 확률 밀도로 대체

- 본 문제의 가장 어려운 부분은 `mus`와 `sigmas`의 적절함 범위를 산정하는 일임
- 이를 위해 고전적 추정 방식으로 `mu`와 `sigma`의 위치를 추정하고, 추정 값의 standard errors를 통해 분포를 파악하려 함
- 분포의 진짜 변수가 $\mu$와 $\sigma$일 때, $\overline\mu$는 샘플 평균인 $m$으로 표기 가능
- $\sigma$의 추정치는 샘플표준편차인 $s$로 표기 가능
- $m$의 standard error는 $s / \sqrt{n}$으로, $s$의 standard error는 $s / \sqrt{2(n-1)}$로 표기 가능

```python
def FindPriorRanges(xs, num_points, num_stderrs = 3):
    
#### m과 s 계산
    n = len(xs)
    m = np.mean(xs)
    s = np.std(xs)
    
#### m과 s의 범위 계산
    stderr_m = s / sqrt(n)
    mus = MakeRange(m, stderr_m)    #### 바로 이어서 정의
    
    stderr_s = s / sqrt(2*(n-1))
    sigmas = MakeRange(s, stderr_s)
    
    return mus, sigmas    
```

- `xs`는 데이터셋, `num_points`는 필요한 값의 갯수, `num_stderrs`는 표준오차 범위를 의미
- 다음은 `MakeRange` 함수 정의임:

```python
def Make Range(estimate, stderr):
    spread = stderr * num_stderrs
    array = np.linspace(estimate - spread,
                        estimate + spread,
                        num_points)
    return array
```
- `estimate - spread`와 `estimate + spread` 간에 `num_points`개의 값으로 이루어진 객체 반환

In [3]:
class Height(thinkbayes.Suite, thinkbayes.Joint):

    
    def __init__(self, mus, sigmas):
        thinkbayes.Suite.__init__(self)

        pairs = [(mu, sigma)
                for mu in mus
                for sigma in sigmas]

        thinkbayes.Suite.__init__(self, pairs)
        
        
    def Likelihood(self, data, hypo):
        x = data
        mu, sigma = hypo
        like = thinkbayes.EvalGaussianPdf(x, mu, sigma)
        return like    

In [4]:
def FindPriorRanges(xs, num_points, num_stderrs = 3):

    n = len(xs)
    m = np.mean(xs)
    s = np.std(xs)

    stderr_m = s / sqrt(n)
    mus = MakeRange(m, stderr_m)    #### 바로 이어서 정의

    stderr_s = s / sqrt(2*(n-1))
    sigmas = MakeRange(s, stderr_s)

    return mus, sigmas
    
    
    def MakeRange(estimate, stderr):
        spread = stderr * num_stderrs
        array = np.linspace(estimate - spread,
                            estimate + spread,
                            num_points)
        return array

___
## 3. 갱신

- 마지막으로 스윗을 만들고 갱신하는 코드는 다음과 같음:

```python
mus, sigmas = FindPriorRanges(xs, num_poiints)
suite = Height(mus, sigmas)
suite.UpdateSet(xs)
print(suite.MaximumLikelihood())
```

- 보통 같은 데이터를 두번 사용하는 방식을 권장하지 않음
- 다만, 본 경우는 사전 분포의 범위를 축소하기 위한 과정일 뿐, 결과에 큰 영향을 끼치지 않음 (has no effect)
- 실제로 사전분포는 `mu`와 `sigma`의 모든 값에 대해 균등분포를 따르지만, 계산 효율성을 위해 필요 없는 값은 무시

___
## 4. CV의 사후 분포

In [6]:
thinkbayes.ReadScale()

AttributeError: module 'thinkbayes' has no attribute 'ReadScale'